# HOLA

Intalamos pandas:

In [156]:
%pip install pandas
%pip install faker

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\pablo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\pablo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [157]:
import pandas as pd 
import cmd, sys
import re
from IPython.display import clear_output
from faker import Factory # type: ignore

In [158]:


class Contact:
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone


    def __str__(self):
        return f"{self.name} | {self.email} | {self.phone}"
    

    def toDataFrame(self):
        return pd.DataFrame([self.__dict__])
       



In [159]:
class CapaPersistencia:
    # .csv
    def __init__(self, filename):
        self.filename = filename
        self.load()
    
    def load(self):
        try:
            self.data = pd.read_csv(self.filename)
        except:
            self.data = pd.DataFrame(columns=["name", "email", "phone"])
            
    def save(self):
        self.data.to_csv(self.filename, index=False)
        

    def insert(self, contact):
        self.data = pd.concat([self.data, contact.toDataFrame()], ignore_index=True)
        self.save()
    
    def delete(self, name):
        self.data = self.data[self.data["name"] != name]
        self.save()
    
    def modify(self, name, email, phone):
        self.data.loc[self.data["name"] == name, "email"] = email
        self.data.loc[self.data["name"] == name, "phone"] = phone
        self.save()

                

In [160]:
class ContactList:
    def __init__(self):
        self.contacts = []
        self.persistencia = CapaPersistencia("contacts.csv")
        self.load()

    def load(self):
        for index, row in self.persistencia.data.iterrows():
            self.contacts.append(Contact(row["name"], row["email"], row["phone"]))

    def save(self):
        for contact in self.contacts:
            self.persistencia.insert(contact)

    def add(self, contact):
        self.contacts.append(contact)
        self.persistencia.insert(contact)
        
    def modify(self, name, contact):
        for c in self.contacts:
            if c.name == name:
                c.email = contact.email
                c.phone = contact.phone
                break
        self.persistencia.modify(contact.name, contact.email, contact.phone)
        
    def delete(self, name):
        for c in self.contacts:
            if c.name == name:
                self.contacts.remove(c)
                self.persistencia.delete(name)
                break

    def clear(self):
        self.contacts = []
        self.persistencia.data = pd.DataFrame(columns=["name", "email", "phone"])
        self.persistencia.save()
    

In [161]:



class SearchEngine:

    def __init__(self, contacts, searchTerm = "", page = 0, regex = False):
        self.contacts = contacts
        self.searchTerm = searchTerm
        self.page = page
        self.items_per_page = 10
        self.total_items = -1
        self.regex = regex

    def search(self):
        found = []

        # term = term.lower()
        # for contact in self.contacts:
        #     phone = contact.phone
        #     phone = phone.lower().replace(" ", "")
        #     #remove country code
           
            
            
        #     if  term in contact.name.lower() or term in contact.email.lower() or term.replace(" ", "") == phone:
        #         found.append(contact)
        
        if self.searchTerm == "":
            return self.contacts
        if self.regex:
            for contact in self.contacts:
                if re.match(self.searchTerm, contact.name) or re.match(self.searchTerm, contact.email) or re.match(self.searchTerm, contact.phone):
                    found.append(contact)
        else:
            term = self.searchTerm.lower()
            for contact in self.contacts:
                phone = contact.phone
                phone = phone.lower().replace(" ", "")
                #remove country code
                if term in contact.name.lower() or term in contact.email.lower() or term.replace(" ", "") == phone:
                    found.append(contact)
        
        self.total_items = len(found)
        return found[self.page * self.items_per_page: (self.page + 1) * self.items_per_page]
    
    def print(self):
        #print("Number of contacts: " + str(len(self.contacts.contacts)))    
       
            
        
        # print("Listing contacts")
        # contacts = self.search_engine.search()
        # for i in range(len(contacts)):
        #     print(str(i) + ". " + str(contacts[i]))
        
       
        # print("Page " + str(page) + " of " + str(self.search_engine.total_items // self.search_engine.items_per_page))
        
        contacts = self.search()
        print("Number of contacts: " + str(self.total_items))
        for i in range(len(contacts)):
            print(str(i) + ". " + str(contacts[i]))
        
        print("Page " + str(self.page) + " of " + str(self.total_items // self.items_per_page))
        
        
        

In [162]:
class ContactShell(cmd.Cmd):
    intro = "Simple contact list"
    prompt = "contact> "
    




    def __init__(self):
        super().__init__()
        self.contacts = ContactList()
        self.search_engine = None
        self.pressed_enter = False
        
    def precmd(self, line: str) -> str:
        if not line:
            return line
        
        if not (line in ["modify", "delete"] and self.search_engine):
            self.clear()
        print(line)
        if line:
            self.pressed_enter = False
        
        if line.split()[0] not in ["search", "list", "regex","modify", "delete"]:
            print(line)
            self.search_engine = None
        
        return super().precmd(line)

    def emptyline(self) -> bool:

        if self.pressed_enter:
            return True
        self.pressed_enter = True
        print("Please enter a command or press enter again to exit.")
        self.do_help("")
        return False
       
    def do_list(self, page = 0):
        'List all contacts'
        #self.clear()
        if not page:
            page = 0
        else:
            try:
                page = int(page)
            except:
                print("Invalid page number")
                return
        self.search_engine = SearchEngine(self.contacts.contacts, page = page)
        self.search_engine.print()
        print()
        print("Type 'list <page>' to see more contacts")
        print("Type 'modify <index>' to modify a contact")
        print("Type 'delete <index>' to delete a contact")
        
        
                
    def do_fakedata(self, args):
        'Generate fake data'
        num = 100
        
        if args:
            try:
                num = int(args)
            except:
                print("Invalid number")
                return
            
        self.contacts.clear()

        fake = Factory.create("es_ES")
        for _ in range(num):
            
            phone = fake.phone_number()
            phone = phone.replace(" ", "").replace("-", "").replace("(", "").replace(")", "")
            
            phone = phone[:3] + " " + phone[3:6] + " " + phone[6:9] + " " + phone[9:]
            
            
            
            
            
            contact = Contact(fake.name(), fake.email(), phone)
            self.contacts.add(contact)

        
                
    def do_search(self, args):
        'Search for a contact'
        
        #args[0] is search term and args[1] is page number(optional)
        
        args = args.split()
        if not args:
            print("usage: search <term> <page>")
            return
        term = args[0]
        page = 0
        if len(args) > 1:
            try:
                page = int(args[1])
            except:
                print("Invalid page number")
                return
        self.search_engine = SearchEngine(self.contacts.contacts, term, page)
        self.search_engine.print()
        print()
        print("Type 'search <term> <page>' to see more contacts")
        print("Type 'modify <index>' to modify a contact")
        print("Type 'delete <index>' to delete a contact")
            
            
    def do_regex(self, args):
        'Search for a contact using regex'
        
        args = args.split()
        if not args:
            print("usage: regex <term> <page>")
            return
        term = args[0]
        page = 0
        if len(args) > 1:
            try:
                page = int(args[len(args) - 1])
                term = " ".join(args[:-1])
            except:
                term = " ".join(args)
        self.search_engine = SearchEngine(self.contacts.contacts, term, page, regex = True)
        self.search_engine.print()
        print()
        print("Type 'regex <term> <page>' to see more contacts")
        print("Type 'modify <index>' to modify a contact")
        print("Type 'delete <index>' to delete a contact")
            
    def do_modify(self, index):
        'Modify a contact'
        if not index:
            print("usage: modify <index>")
            return
        if not self.search_engine:
            print("Please search for a contact first use 'search' or 'list'")
            return
        try:
            index = int(index)
        except:
            print("Invalid index")
            return
        contacts = self.search_engine.search()
        if index < 0 or index >= len(contacts):
            print("Invalid index")
            return
        contact = contacts[index]
        print("Contact found: " + str(contact))
        print("Enter new data")
        email = input("Email (blank to keep current) : ")
        phone = input("Phone (blank to keep current): ")
        
        if not email:
            email = contact.email
        if not phone:
            phone = contact.phone
        
        
        contact = Contact(contact.name, email, phone)
        self.contacts.modify(contact.name, contact)
        
    def do_delete(self, index):
        'Delete a contact'
        if not index:
            print("usage: delete <index>")
            return
        if not self.search_engine:
            print("Please search for a contact first use 'search' or 'list'")
            return
        try:
            index = int(index)
        except:
            print("Invalid index")
            return
        contacts = self.search_engine.search()
        if index < 0 or index >= len(contacts):
            print("Invalid index")
            return
        contact = contacts[index]
        print("Contact found: " + str(contact))
        confirm = input("Are you sure you want to delete this contact? (yes/no) ")
        if confirm == "yes":
            self.contacts.delete(contact.name)
        else:
            print("Contact not deleted")
        
        
        

    def do_add(self, args):
        'Add a contact'
        if not args:
            print("usage: add <name> <email>")
            return
        name, email = args.split()
        contact = Contact(name, email)
        self.contacts.add(contact)

    def do_exit(self, args):
        'Exit the program'
        return True
    
    
    def default(self, line: str) -> None:
        print(f"*** Unknown syntax: {line}")
        self.do_help("")
    
    def clear(self): 
        clear_output(wait = True)
    

In [163]:
class CLI:
    def __init__(self):
        
        self.contacts = ContactList()
        
        self.engine = SearchEngine(self.contacts.contacts)

    def run(self):
        while True:
            print("1. Add contact")
            print("2. Search contact")
            print("3. Exit")
            option = input("Choose an option: ")
            if option == "1":
                self.add_contact()
            elif option == "2":
                self.search_contact()
            elif option == "3":
                break
            else:
                print("Invalid option")
                
    def add_contact(self):
        name = input("Name: ")
        email = input("Email: ")
        phone = input("Phone: ")
        contact = Contact(name, email, phone)
        test = contact.toDataFrame()
        
        
        self.contacts.add(contact)
 
    
    def search_contact(self):
        term = input("Search term: ")
        found = self.engine.search(term)
        for contact in found:
            print(contact)
            


In [164]:

if __name__ == '__main__': 
    shell = ContactShell()
    shell.cmdloop()
    #cli = CLI()
    #cli.run()


search hola
Number of contacts: 1
0. Abel Suárez Miralles | hola | +34 981 671 666
Page 0 of 0

Type 'search <term> <page>' to see more contacts
Type 'modify <index>' to modify a contact
Type 'delete <index>' to delete a contact
Please enter a command or press enter again to exit.

Documented commands (type help <topic>):
add  delete  exit  fakedata  help  list  modify  regex  search

